# Построение индикатора по результатам сентимент-анализа с помощью RuBERT

In [1]:
import pandas as pd

In [2]:
df_ec=pd.read_excel('EC_RuBERT-base_sentiment_classifier_3_labels_7000_2_epoch.xlsx')
df_cb=pd.read_excel('CB_RuBERT-base_sentiment_classifier_3_labels_7000_2_epoch.xlsx')

In [3]:
data = pd.DataFrame()
data['date'] = pd.date_range(start='2014-1-1', end='2024-12-31')
data['year_month'] = [str(data.iloc[i]['date'])[:7] for i in range(data.shape[0])]
data['year'] = [str(data.iloc[i]['date'])[:4] for i in range(data.shape[0])]
data.set_index('date', inplace = True, drop = False)
data['num_ec'] = 0
data['1_ec'] = 0
data['-1_ec'] = 0
data['sum_mood_ec'] = 0
data['num_cb'] = 0
data['1_cb'] = 0
data['-1_cb'] = 0
data['sum_mood_cb'] = 0
for i in range(df_ec.shape[0]):
  data.loc[df_ec['date'][i], 'num_ec'] = data['num_ec'][df_ec['date'][i]] + 1
  data.loc[df_ec['date'][i], '1_ec'] = data['1_ec'][df_ec['date'][i]] + (df_ec['predict'][i] == 2)
  data.loc[df_ec['date'][i], '-1_ec'] = data['-1_ec'][df_ec['date'][i]] + (df_ec['predict'][i] == 0)
  data.loc[df_ec['date'][i], 'sum_mood_ec'] = data['sum_mood_ec'][df_ec['date'][i]] + df_ec['Ppos-Pneg'][i]
for i in range(df_cb.shape[0]):
  data.loc[df_cb['date'][i], 'num_cb'] = data['num_cb'][df_cb['date'][i]] + 1
  data.loc[df_cb['date'][i], '1_cb'] = data['1_cb'][df_cb['date'][i]] + (df_cb['predict'][i] == 2)
  data.loc[df_cb['date'][i], '-1_cb'] = data['-1_cb'][df_cb['date'][i]] + (df_cb['predict'][i] == 0)
  data.loc[df_cb['date'][i], 'sum_mood_cb'] = data['sum_mood_cb'][df_cb['date'][i]] + df_cb['Ppos-Pneg'][i]

C:\Users\Anastasia\AppData\Local\Temp\ipykernel_11348\1898322403.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.8173546195030212' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[df_ec['date'][i], 'sum_mood_ec'] = data['sum_mood_ec'][df_ec['date'][i]] + df_ec['Ppos-Pneg'][i]
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_11348\1898322403.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.7963014245033264' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[df_cb['date'][i], 'sum_mood_cb'] = data['sum_mood_cb'][df_cb['date'][i]] + df_cb['Ppos-Pneg'][i]


## Построение индикатора доверия к ЦБ с ежедневной частотой

In [6]:
data_days = data.copy()
data_days['mood_ec']=data_days['sum_mood_ec']/data_days['num_ec']
data_days['mood_cb']=data_days['sum_mood_cb']/data_days['num_cb']
data_days['mood_cb_without_mood_ec']=data_days['mood_cb']-data_days['mood_ec'] #значения могут находиться на отрезке [-2,2]
data_days['indicator']=(data_days['mood_cb_without_mood_ec']+2)/4 #приведение множества возможных значений индикатора к отрезку [0,1]
data_days['indicator2']=((data_days['1_cb']-data_days['-1_cb'])/data_days['num_cb']-(data_days['1_ec']-data_days['-1_ec'])/data_days['num_ec'])/4+0.5
#indicator2 - альтернативный способ расчёта индикатора на основе меток классов

## Построение индикатора доверия к ЦБ с ежеквартальной частотой

In [8]:
data_month = data.drop(['date', 'year'], axis=1).groupby(['year_month']).agg('sum')
data_month['mood_ec']=data_month['sum_mood_ec']/data_month['num_ec']
data_month['mood_cb']=data_month['sum_mood_cb']/data_month['num_cb']
data_month['mood_cb_without_mood_ec']=data_month['mood_cb']-data_month['mood_ec'] #значения могут находиться на отрезке [-2,2]
data_month['indicator']=(data_month['mood_cb_without_mood_ec']+2)/4 #приведение множества возможных значений индикатора к отрезку [0,1]
data_month['indicator2']=((data_month['1_cb']-data_month['-1_cb'])/data_month['num_cb']-(data_month['1_ec']-data_month['-1_ec'])/data_month['num_ec'])/4+0.5
#indicator2 - альтернативный способ расчёта индикатора на основе меток классов

## Построение индикатора доверия к ЦБ с ежегодной частотой

In [10]:
data_year = data.drop(['date', 'year_month'], axis=1).groupby(['year']).agg('sum')
data_year['mood_ec']=data_year['sum_mood_ec']/data_year['num_ec']
data_year['mood_cb']=data_year['sum_mood_cb']/data_year['num_cb']
data_year['mood_cb_without_mood_ec']=data_year['mood_cb']-data_year['mood_ec'] #значения могут находиться на отрезке [-2,2]
data_year['indicator']=(data_year['mood_cb_without_mood_ec']+2)/4 #приведение множества возможных значений индикатора к отрезку [0,1]
data_year['indicator2']=((data_year['1_cb']-data_year['-1_cb'])/data_year['num_cb']-(data_year['1_ec']-data_year['-1_ec'])/data_year['num_ec'])/4+0.5
#indicator2 - альтернативный способ расчёта индикатора на основе меток классов

In [11]:
data_days.to_csv('indicator_d_RuBERT_7000.csv')
data_month.to_csv('indicator_m_RuBERT_7000.csv')
data_year.to_csv('indicator_y_RuBERT_7000.csv')